In [ ]:
# Imports
import pandas as pd

In [ ]:


# Read the ctms to a dataframe
column_names=["Filename", "segment", "start", "duration", "token"]
gold_ctms_file = "../data/ctm_from_docker_wholesale"
gold_ctms_df = pd.read_csv(gold_ctms_file, sep=' ', names=column_names, index_col=False, engine='python')

created_ctms_file = "../data/ctm_to_Marko_Jouste"
created_ctms_df = pd.read_csv(created_ctms_file, sep=' ', names=column_names, index_col=False, engine='python')


In [ ]:
# Create end-time column
'''df['c'] = df.apply(lambda row: row.a + row.b, axis=1)
If you get the SettingWithCopyWarning you can do it this way also:

fn = lambda row: row.a + row.b # define a function for the new column
col = df.apply(fn, axis=1) # get column data with an index
df = df.assign(c=col.values) # assign values to column 'c'
'''
gold_ctms_df["end"] = gold_ctms_df.apply(lambda row: row.start + row.duration, axis=1)
created_ctms_df["end"] = created_ctms_df.apply(lambda row: row.start + row.duration, axis=1)

In [ ]:
# examine data
gold_ctms_df.head()


In [ ]:
created_ctms_df.head()


In [ ]:

# Iterate file by file
list_of_filenames = gold_ctms_df["Filename"].unique().tolist()
number_of_files = len(list_of_filenames)

total_correct_empty_frames = []
total_correct_token_frames = []
total_incorrect_token_frames = []
total_number_of_frames = []

for filename in list_of_filenames:
    correct_empty_frames = 0
    correct_token_frames = 0
    incorrect_token_frames = 0
    number_of_frames = 0
    
    df_current_gold_ctm = gold_ctms_df.loc[gold_ctms_df['Filename'] == filename][["start", "end", "token"]]
    df_current_created_ctm = created_ctms_df.loc[created_ctms_df['Filename'] == filename][["start", "end", "token"]]
    
    # Iterate by frame (this could be much faster, but this is lot more simpler)
    endtime = int(df_current_gold_ctm["end"].max() * 1000) # in ms
    framerate = 10 # in ms
    for i in range(0, endtime, framerate):
        current_time = i / 1000
        gold_tokens_set = set(df_current_gold_ctm.loc[(df_current_gold_ctm["start"] <= current_time) & (df_current_gold_ctm["end"] >= current_time)]["token"].unique())
        created_tokens_set = set(df_current_created_ctm.loc[(df_current_created_ctm["start"] <= current_time) & (df_current_created_ctm["end"] >= current_time)]["token"].unique())
       #%% 
        if (0 == len(gold_tokens_set)) and (0 == len(created_tokens_set)):
            correct_empty_frames += 1
        
        elif not gold_tokens_set.isdisjoint(created_tokens_set):
            correct_token_frames += 1
        else:
            incorrect_token_frames += 1
        
        number_of_frames += 1
            
    total_correct_token_frames.append(correct_token_frames)
    total_correct_empty_frames.append(correct_empty_frames)
    total_incorrect_token_frames.append(incorrect_token_frames)
    total_number_of_frames.append(number_of_frames)
    #%%


In [ ]:
current_time = 1.5 
filename = "N_Tsd_Cd_m1"
df_current_gold_ctm = gold_ctms_df.loc[gold_ctms_df['Filename'] == filename][["start", "end", "token"]]
gold_tokens_set = set(df_current_gold_ctm.loc[(df_current_gold_ctm["start"] <= current_time) & (df_current_gold_ctm["end"] >= current_time)]["token"].unique())
